In [ ]:
from langchain_openai.llms import OpenAI

llm = OpenAI()

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Responda a seguinte pergunta do usuário:
    {pergunta}
    """
)

In [ ]:
print(prompt_template.format(pergunta="O que é um SaaS?"))

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
    {pergunta}
    """
)

In [ ]:
print(prompt_template.format(pergunta="O que é um SaaS?", n_palavras=10))

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
    {pergunta}
    """, partial_variables={"n_palavras": 10}
)

In [ ]:
print(prompt_template.format(pergunta="O que é um langchain?"))

### Utilizando Múltiplos Prompts

In [ ]:
from langchain.prompts import PromptTemplate

template_word_count = PromptTemplate.from_template(
    """
    Responda a seguinte pergunta do usuário em até {n_palavras} palavras.
    """
)

template_line_count = PromptTemplate.from_template(
    """
    Responda a seguinte pergunta do usuário em até {n_linhas} linhas.
    """
)

template_idioma = PromptTemplate.from_template(
    """
    Retorne a resposta no {idioma}.
    """
)


template_final = (template_word_count + template_line_count + template_idioma +
                  "Responda a pergunta seguindo as instruções {pergunta}")

In [ ]:
print(template_final)

In [ ]:
prompt_final = template_final.format(pergunta="O que é um SaaS?", n_palavras=30, n_linhas=3, idioma="Português")
llm.invoke(prompt_final)

In [ ]:
print(prompt_final)

### Templates para Chat

In [ ]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template("Essa é minha dúvida: {duvida}")
chat_template.format_messages(duvida="O que é um SaaS?")

In [ ]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um assistente irônico e se chama {nome_assistente}"),
        ("human", "Olá, como vai?"),
        ("ai", "Estou bem, como posso lhe ajudar?"),
        ("human", "{pergunta}")
    ]
)

In [ ]:
chat_template

In [ ]:
chat_template.format_messages(nome_assistente="ChatBot", pergunta="Qual seu nome?")

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

chat.invoke(chat_template.format_messages(nome_assistente="ChatBot", pergunta="Qual seu nome?"))

### Few Shot Prompting

In [ ]:
exemplos = [
    {"pergunta": "Qual é a maior montanha do mundo, o Monte Everest ou o K2?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Qual é a altura do Monte Everest? 
Resposta intermediária: O Monte Everest tem 8.848 metros de altura. 
Pergunta de acompanhamento: Qual é a altura do K2? 
Resposta intermediária: O K2 tem 8.611 metros de altura. 
Então a resposta final é: Monte Everest 
""", 
    }, 
    {"pergunta": "Quem nasceu primeiro, Charles Darwin ou Albert Einstein?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quando nasceu Charles Darwin? 
Resposta intermediária: Charles Darwin nasceu em 12 de fevereiro de 1809. 
Pergunta de acompanhamento: Quando nasceu Albert Einstein? 
Resposta intermediária: Albert Einstein nasceu em 14 de março de 1879. 
Então a resposta final é: Charles Darwin 
""", 
    }, 
    {"pergunta": "Quem foi o pai de Napoleão Bonaparte?",
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi Napoleão Bonaparte? 
Resposta intermediária: Napoleão Bonaparte foi um líder militar e imperador francês. 
Pergunta de acompanhamento: Quem foi o pai de Napoleão Bonaparte? 
Resposta intermediária: O pai de Napoleão Bonaparte foi Carlo Buonaparte. 
Então a resposta final é: Carlo Buonaparte 
""", 
    },
    {"pergunta": "Os filmes 'O Senhor dos Anéis' e 'O Hobbit' foram dirigidos pelo mesmo diretor?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem dirigiu 'O Senhor dos Anéis'? 
Resposta intermediária: 'O Senhor dos Anéis' foi dirigido por Peter Jackson. 
Pergunta de acompanhamento: Quem dirigiu 'O Hobbit'? 
Resposta intermediária: 'O Hobbit' também foi dirigido por Peter Jackson. 
Então a resposta final é: Sim 
""",
    },
]

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["pergunta", "resposta"],
    template="Pergunta {pergunta}\n{resposta}"
)

example_prompt.format(**exemplos[0])

In [ ]:
prompt = FewShotPromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt,
    suffix="Pergunta: {input}",
    input_variables=["input"]
)

In [ ]:
print(prompt.format(input="Quem é melhor, Messi ou Cristiano Ronaldo?"))

In [ ]:
llm.invoke(prompt.format(input="Quem fez mais gols, Messi ou Cristiano Ronaldo?"))

### Few Shot Prompting em Chat

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

In [ ]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{pergunta}"),
     ("ai", "{resposta}")]
)

print(example_prompt.format_messages(**exemplos[0]))

In [ ]:
few_shot_template = FewShotChatMessagePromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt
)

prompt_final = ChatPromptTemplate.from_messages([
    few_shot_template,
    ("human", "{input}")
])

prompt = prompt_final.format_messages(input="Quem fez mais gols, Messi ou Cristiano Ronaldo?")
# few_shot_template.format_messages()

In [ ]:
chat.invoke(prompt)